# Scraping Used Car Web Data: Case Study tucarrro.com (Colab version)
[Author: Elias Buitrago Bolivar](https://github.com/ebuitrago?tab=repositories)

This jupyter notebook depicts a python based web scraping  algorithm to obtain data to train a price car prediction machine learning algorithm. Used cars web data are extracted from [Tu Carro](www.tucarro.com.co). The code presented here is functional and was tested by scraping real data. This code version is compatible with Colab.
_Updated: Feb 18, 2024_


## Install required libraries

In [1]:
!pip install lxml
!pip install scrapy
!pip3 install requests-html
!pip3 install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.8/287.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%%shell
# Install chromedriver
# Credits: https://medium.com/@MinatoNamikaze02/running-selenium-on-google-colab-a118d10ca5f8
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.62/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver

pip install selenium chromedriver_autoinstaller

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [736 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,568 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:13 http://security.ubuntu.

### Web Scraping Used Car Sales Data
This section explains the web scraping process implemented to obtain the data from the used car sales web site [Tu Carro](www.tucarro.com.co).

In [3]:
!pip install undetected_chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for undetected_chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47046 sha256=27c3099286d99bf864ca1711526e39d5cc03aa4eb8a542038ba6b298eba7db2d
  Stored in directory: /root/.cache/pip/wheels/cf/a1/db/e1275b6f7259aacd6b045f8bfcb1fcbc93827a3916ba55d5b7
Successfully built undetected_chromedriver


## Import required libraries


---

In [4]:
'''
credits:
https://github.com/googlecolab/colabtools/issues/3347
https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
Sept 19, 2023
'''

#
!pip3 install chromedriver-autoinstaller

In [5]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import chromedriver_autoinstaller

## Setup chrome and chrome driver


---



In [6]:
# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# # set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/122/chromedriver'


## Section to declare functions

---



### Function scrapebyPages

In [7]:
def scrapebyPages(brand,model,min, max):
  #Range of pages from the total search to scrape in.
  #It is recomended to cover a range of one hundred pages in each iteration of this section.
  data = pd.DataFrame()
  for i in range(min,max):

      print(f'************************************')
      print(f'WEB SCRAPING FROM SEARCH PAGE #{i}')
      pag = i
      url = f'https://vehiculos.tucarro.com.co/{brand}/{model}/_Desde_{49*i}_NoIndex_True'
      # url = f'https://vehiculos.tucarro.com.co/maxda-cx-30_Desde_{49*i}_NoIndex_True'
      # print(url)

      driver = webdriver.Chrome(options=chrome_options)
      driver.get(url)
      driver.implicitly_wait(10)
      html = driver.page_source
      soup = bs(html,'lxml')

      #Get href
      links = gethref(soup)

      #Scrapping
      # for i in range(0,5):
      #   soup = scrapper(links[i])
      #   # print(soup)

      p = []
      # cols = ['car_model','price','year_model','kms']
      # data = pd.DataFrame(columns=cols)
      #Scrapping a los inmuebles filtrados
      # for i in range(len(links)):
      for i in range(0,len(links)):
          print('Scrapping', i, '/', len(links), '...')
          p.append(scrapper(links[i]))
          print(f'Este es el valor de p[i]: {p[i]}')

      # #append list to DataFrame
      data = data.append(p, ignore_index = True)
      # print(f'This is the dataset:\n {data}')

  #Close the web browser tab
  driver.close()

  # quit the driver
  driver.quit()

  return data

### Function gethref

In [8]:
#Function to get 'href' from each article item
def gethref(soup):

    links = []
    for link in soup.findAll('a'):
      url_car = link.get('href')
      if 'MCO-' in url_car:
        # print(url_car)          %Print each car url as a validity test
        links.append(url_car)

    print("Href obtained: ", len(links))

    # for article in soup.find_all('article'):
    #     url = article.find('a', href=True)
    #     if url:
    #         link = url['href']
    #         links.append(link)
    # print("Href obtained: ", len(links))

    return links
    # return

### Function scrapper

In [9]:
#Function to call housing_features routine on each href
def scrapper(url_car):

    # set up the webdriver
    driver = webdriver.Chrome(options=chrome_options)

    # Scrape
    driver.get(url_car)
    driver.implicitly_wait(10)
    html=driver.page_source

    #Obtaining the html from the web page after applying Selenium
    soup = bs(html,'lxml')

    #Create a list to store info obtained from one particular property
    features = []

    #Applying function to obtain variables defined from one particular property
    features = extract_cars_features(soup)

    #Close the web browser tab
    driver.close()

    # quit the driver
    driver.quit()

    return(features)

### Function extract_cars_features

In [10]:
# Version 1.0
def extract_cars_features(soup):

  features_list = []

  # car_name
  try:
    car_name = soup.find('h1',{'class': 'ui-pdp-title'}).text
    features_list.append(car_name)
    # print(f"Car's name is: {car_name}")
  except:
    car_name = ' '
    features_list.append(car_name)

  # price
  try:
    price=soup.find('div',{'class': 'ui-pdp-price__second-line'}).text
    features_list.append(price)
    # print(f"Car's price is: {price}")
  except:
    price = 0
    features_list.append(price)

  # year_car
  try:
    year_kms_datePub = soup.find('div',{'class': 'ui-pdp-header__subtitle'}).text.split(' ')
    year = year_kms_datePub[0]
    features_list.append(year)
  except:
    year = 0
    features_list.append(year)

  # kms
  try:
    year_kms_datePub = soup.find('div',{'class': 'ui-pdp-header__subtitle'}).text.split(' ')
    kms = year_kms_datePub[2]
    features_list.append(kms)
  except:
    kms = 0
    features_list.append(kms)
  # print(f"Kms: {kms}")

  # # date_publication_1
  # datePub = ' '.join(year_kms_datePub[7:])
  # features_list.append(datePub)
  # # print(f"Publication date: {datePub}")


  # print(features_list)


  return features_list

## Start scraping

---

In [12]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [13]:
"""
 The input parameters for the 'scrapebyPages' function are: Brand name, Car model
 name. Be careful to write the brand and model names exactly as they are in tucarro.com.
 The third input parameter is the initial results page (always initialize to 1)
 and the fourth input parameter is the final results page you want to download data from;
 this parameter depends on the amount of results pages your car returns
 for the brand and model you want to get data from. So, it is recommended to search
 the web portal first to find out how many pages of results you can get
 for the car you want to get data from.
"""

car_brand = 'kia'   # Brand car name. Ej: chevrolet, renault, kia.
car_model = 'rio'        # Model car name. Ej: duster, onix, rio.
data = scrapebyPages(car_brand,car_model,1,2)
# scrapebyPages(1,2)

************************************
WEB SCRAPING FROM SEARCH PAGE #1
Href obtained:  48
Scrapping 0 / 48 ...
Este es el valor de p[i]: ['Kia Rio 1.4 Vibrant', '$65.900.000', '2023', '37.600']
Scrapping 1 / 48 ...
Este es el valor de p[i]: ['Kia Rio 1.4 4 p', '$46.500.000', '2014', '73.435']
Scrapping 2 / 48 ...
Este es el valor de p[i]: ['Kia Rio 1.4 ZENITH', '$61.500.000', '2019', '44.000']
Scrapping 3 / 48 ...
Este es el valor de p[i]: ['Kia Rio Ex 1 4 Tp 1.4 4p 2ab 2015', '$40.990.000', '2015', '110.540']
Scrapping 4 / 48 ...
Este es el valor de p[i]: ['Kia Rio 1.4 Vibrant At', '$58.600.000', '2019', '54.105']
Scrapping 5 / 48 ...
Este es el valor de p[i]: ['Kia Rio FE 1.4 AUT', '$54.500.000', '2018', '61.187']
Scrapping 6 / 48 ...
Este es el valor de p[i]: ['Kia Rio 1.4 Vibrant Hb 4 p', '$67.900.000', '2023', '28.512']
Scrapping 7 / 48 ...
Este es el valor de p[i]: ['Kia Rio Ub Ex-1.4 Mec Hb Modelo 2017 45677', '$47.300.000', '2017', '104.632']
Scrapping 8 / 48 ...
Este es el valo

<ipython-input-7-d9697e0ef0fd>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(p, ignore_index = True)


In [14]:
cols = ['car_model','price','year_model','kms']
data.columns = cols
print(data.shape)
data.head()

(48, 4)


,car_model,price,year_model,kms
0,Kia Rio 1.4 Vibrant,$65.900.000,2023,37.600
1,Kia Rio 1.4 4 p,$46.500.000,2014,73.435
2,Kia Rio 1.4 ZENITH,$61.500.000,2019,44.000
3,Kia Rio Ex 1 4 Tp 1.4 4p 2ab 2015,$40.990.000,2015,110.540
4,Kia Rio 1.4 Vibrant At,$58.600.000,2019,54.105


In [16]:
saved_name=f'usedCarsCol_{car_model}_120324_small.csv'
data.to_csv(saved_name, encoding='utf-8', index=False)

### Testing code for scraping only one page

In [ ]:
#*****************************
#Code for testing in one page
#*****************************

# # set the target URL
# pag = 1
# url = f'https://www.fincaraiz.com.co/apartamentos-casas/venta/bogota/bogota-dc?pagina={pag}'

# # set up the webdriver
# driver = webdriver.Chrome(options=chrome_options)

# print(url)
# driver.get(url)
# driver.implicitly_wait(10)
# html = driver.page_source
# soup = bs(html,'lxml')

# # quit the driver
# driver.quit()

# links = []
# links = gethref(soup)
# links

# aux = []
# cols = ['habitaciones','baños','parqueaderos','area_construida','area_privada','estrato','estado','antiguedad',
#         'administracion','precio_m2', 'Ascensor', 'Circuito cerrado de TV',
#        'Parqueadero Visitantes', 'Portería / Recepción', 'Zonas Verdes', 'Salón Comunal', 'Balcón',
#        'Barra estilo americano', 'Calentador', 'Chimenea', 'Citófono', 'Cocina Integral', 'Terraza',
#        'Vigilancia', 'Parques cercanos', 'Estudio', 'Patio', 'Depósito / Bodega', 'nombre','ubicacion','precio']
# data = pd.DataFrame(columns=cols)

# #Remove "Proyectos de vivienda"
# url_inmuebles = []
# url_inmuebles = remove_proyectos(links)

# #Scrapping
# p = []
# #Scrapping a los inmuebles filtrados
# for i in range(len(url_inmuebles)):
#   print('Scrapping', i, '/', len(url_inmuebles), '...')
#   p.append(scrapper(url_inmuebles[i]))
#   print(p[i])

#   #append list to DataFrame
#   data.loc[len(data)] = p[i]

## Referencias
---



https://github.com/kiteco/kite-python-blog-post-code/blob/master/Web%20Scraping%20Tutorial/script.py

https://medium.com/geekculture/scrappy-guide-to-web-scraping-with-python-475385364381

https://stackoverflow.com/questions/47730671/python-3-using-requests-does-not-get-the-full-content-of-a-web-page